Copyright © 2022-2023 HQS Quantum Simulations GmbH. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except
in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.

# Measuring qubits in qoqo

This notebook is designed to demonstrate the use of measurements in qoqo. We will look at several examples of measuring qubits, from single and multi-qubit registers. To learn about the effect of measurement, we will look at the state vectors before and after measurement. 

In [ ]:
from qoqo_quest import Backend
from qoqo import Circuit
from qoqo import operations as ops 

## Measuring a single qubit

Here we first prepare the qubit in a superposition state, 
\begin{equation}
|+ \rangle = \frac{1}{\sqrt{2}} \big ( |0 \rangle + |1 \rangle \big ).
\end{equation}
We look at the state after preparation, then do a measurement in the Z basis, and finally look again at the state after measurement. 

We see that the state after measurement has been projected into the state either $|0 \rangle$ or $|1\rangle$, consistently with the measurement outcome. Running this code many times should result in a random distribution of 'True' and 'False' outcomes.

In [ ]:
state_init = Circuit()
# prepare |+> state
state_init += ops.Hadamard(qubit=0)  

# write state before measuring to readout register 'psi_in'
read_input = Circuit()
read_input += ops.DefinitionComplex(name='psi_in', length=2, is_output=True)
read_input += ops.PragmaGetStateVector(readout='psi_in', circuit=Circuit())

# measure qubit in Z basis and write result to classical register 'M1'
meas_circ = Circuit()
meas_circ += ops.DefinitionBit(name='M1', length=1, is_output=True)
meas_circ += ops.MeasureQubit(qubit=0, readout='M1', readout_index=0)

# write state after measuring to readout register 'psi_out'
read_output = Circuit()
read_output += ops.DefinitionComplex(name='psi_out', length=2, is_output=True)
read_output += ops.PragmaGetStateVector(readout='psi_out', circuit=Circuit())

# put each step of the circuit together
circuit = state_init + read_input + meas_circ + read_output

# run the circuit and collect output
backend = Backend(number_qubits=1)
(result_bit_registers, result_float_registers, result_complex_registers) \
        = backend.run_circuit(circuit)

print('Input state: \n', result_complex_registers['psi_in'][0], '\n')
print('Measurement result: ', result_bit_registers['M1'][0][0], '\n')
print('State after measurement: \n', result_complex_registers['psi_out'][0])

## Measuring a single qubit in the X basis

Instead of measuring in the $Z$ basis, we can measure the qubit in the $X$ basis by performing a Hadamard operator before the measurement. 

This time we see that the measurement result is always 'False', since we are measuring the $|+ \rangle$ state in the $X$ basis, and it is an $X$ eigenvector of the $X$ operator. 

In [ ]:
# add Hadamard operator to change from Z to X basis
meas_X_circ = Circuit()
meas_X_circ += ops.DefinitionBit(name='M1', length=1, is_output=True)
meas_X_circ += ops.Hadamard(qubit=0)
meas_X_circ += ops.MeasureQubit(qubit=0, readout='M1', readout_index=0)

# perform additional Hadamard after measurement to readout in Z basis
read_output = Circuit()
read_output += ops.DefinitionComplex(name='psi_out', length=2, is_output=True)
read_output += ops.Hadamard(qubit=0)
read_output += ops.PragmaGetStateVector(readout='psi_out', circuit=Circuit())

circuit = state_init + read_input + meas_X_circ + read_output

# run the circuit and collect output
backend = Backend(number_qubits=1)
(result_bit_registers, result_float_registers, result_complex_registers) \
        = backend.run_circuit(circuit)

print('Input state: \n', result_complex_registers['psi_in'][0], '\n')
print('Measurement result: ', result_bit_registers['M1'][0][0], '\n')
print('State after measurement: \n', result_complex_registers['psi_out'][0])

## Measuring a multi-qubit register

Here we first prepare a multi-qubit register and demonstrate how it is possible to measure the entire register. As an example we prepare the multi-qubit register in the state, 
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} |010 \rangle + \frac{i}{\sqrt{2}} |101 \rangle.
\end{equation}

After preparation we read out the simulated state, before measurement. Next we measure each qubit of the state, and finally we readout out the post-measurement state. 

In [ ]:
number_of_qubits = 3

state_init = Circuit()
state_init += ops.PauliX(qubit=1) 
state_init += ops.Hadamard(qubit=0) 
state_init += ops.CNOT(control=0, target=1) 
state_init += ops.CNOT(control=0, target=2)
state_init += ops.SGate(qubit=0)

# write state before measuring to readout register 'psi_in'
read_input = Circuit()
read_input += ops.DefinitionComplex(name='psi_in', length=2**number_of_qubits,
                                    is_output=True)
read_input += ops.PragmaGetStateVector(readout='psi_in', circuit=Circuit())

# measure qubits in Z basis and write result to classical register 'M1M2M3'
meas_circ = Circuit()
meas_circ += ops.DefinitionBit(name='M1M2M3', length=3, is_output=True)
meas_circ += ops.MeasureQubit(qubit=0, readout='M1M2M3', readout_index=0)
meas_circ += ops.MeasureQubit(qubit=1, readout='M1M2M3', readout_index=1)
meas_circ += ops.MeasureQubit(qubit=2, readout='M1M2M3', readout_index=2)

# write state after measuring to readout register 'psi_out'
read_output = Circuit()
read_output += ops.DefinitionComplex(name='psi_out', length=2**number_of_qubits,
                                    is_output=True)
read_output += ops.PragmaGetStateVector(readout='psi_out', circuit=Circuit())


circuit = state_init + read_input + meas_circ + read_output

# run the circuit and collect output
backend = Backend(number_qubits=number_of_qubits)
(result_bit_registers, result_float_registers, result_complex_registers) \
        = backend.run_circuit(circuit)

print('Input state: \n', result_complex_registers['psi_in'][0], '\n')
print('Measurement results: ', result_bit_registers['M1M2M3'][0], '\n')
print('State after measurement: \n', result_complex_registers['psi_out'][0])

## Measuring one qubit from a multi-qubit register

Measuring only a single qubit from a multi-qubit register is an almost identical process to measuring the entire register, except we only add a single measurement in this case. 

Here we again prepare the input state, 
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} |010 \rangle + \frac{i}{\sqrt{2}} |101 \rangle.
\end{equation}

After preparation we read out the simulated state, before measurement. Next we measure the first qubit of the state, and finally we readout out the post-measurement state.

In [ ]:
number_of_qubits = 3

state_init = Circuit()
state_init += ops.PauliX(qubit=1) 
state_init += ops.Hadamard(qubit=0) 
state_init += ops.CNOT(control=0, target=1) 
state_init += ops.CNOT(control=0, target=2)
state_init += ops.SGate(qubit=0)

# write state before measuring to readout register 'psi_in'
read_input = Circuit()
read_input += ops.DefinitionComplex(name='psi_in', length=2**number_of_qubits,
                                    is_output=True)
read_input += ops.PragmaGetStateVector(readout='psi_in', circuit=Circuit())

# measure qubit in Z basis and write result to classical register 'M1'
meas_circ = Circuit()
meas_circ += ops.DefinitionBit(name='M1', length=1, is_output=True)
meas_circ += ops.MeasureQubit(qubit=0, readout='M1', readout_index=0)


# write state after measuring to readout register 'psi_out'
read_output = Circuit()
read_output += ops.DefinitionComplex(name='psi_out', length=2**number_of_qubits,
                                    is_output=True)
read_output += ops.PragmaGetStateVector(readout='psi_out', circuit=Circuit())


circuit = state_init + read_input + meas_circ + read_output

# run the circuit and collect output
backend = Backend(number_qubits=number_of_qubits)
(result_bit_registers, result_float_registers, result_complex_registers) \
        = backend.run_circuit(circuit)

print('Input state: \n', result_complex_registers['psi_in'][0], '\n')
print('Measurement results: ', result_bit_registers['M1'][0], '\n')
print('State after measurement: \n', result_complex_registers['psi_out'][0])